In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import time
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import struct
import os

# ==================== DEVICE SETUP ====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==================== CUSTOM MNIST LOADER ====================
def load_mnist_images(filepath):
    """Charge les images MNIST depuis un fichier .ubyte"""
    with open(filepath, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack('>4I', f.read(16))
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(num_images, rows, cols)

def load_mnist_labels(filepath):
    """Charge les labels MNIST depuis un fichier .ubyte"""
    with open(filepath, 'rb') as f:
        magic, num_labels = struct.unpack('>2I', f.read(8))
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data

class MNISTDataset(Dataset):
    """Dataset personnalisé pour MNIST"""
    def __init__(self, images, labels, transform=None):
        self.images = torch.FloatTensor(images).unsqueeze(1) / 255.0
        self.labels = torch.LongTensor(labels)
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ==================== DATA LOADING ====================
print("\nChargement du dataset MNIST...")

transform = transforms.Compose([
    transforms.Normalize((0.1307,), (0.3081,))
])

# Essayer de charger depuis les fichiers Kaggle
kaggle_path = '/kaggle/input/datasets/mnist-dataset'
local_path = './data'

try:
    # Essayer d'abord le chemin Kaggle
    train_images = load_mnist_images(f'{kaggle_path}/train-images-idx3-ubyte')
    train_labels = load_mnist_labels(f'{kaggle_path}/train-labels-idx1-ubyte')
    test_images = load_mnist_images(f'{kaggle_path}/t10k-images-idx3-ubyte')
    test_labels = load_mnist_labels(f'{kaggle_path}/t10k-labels-idx1-ubyte')
    
    train_dataset = MNISTDataset(train_images, train_labels, transform=transform)
    test_dataset = MNISTDataset(test_images, test_labels, transform=transform)
    print("✓ Dataset chargé depuis Kaggle (/kaggle/input/mnist-dataset/mnist)")
except FileNotFoundError:
    try:
        # Essayer le chemin local
        train_images = load_mnist_images(f'{local_path}/train-images-idx3-ubyte')
        train_labels = load_mnist_labels(f'{local_path}/train-labels-idx1-ubyte')
        test_images = load_mnist_images(f'{local_path}/t10k-images-idx3-ubyte')
        test_labels = load_mnist_labels(f'{local_path}/t10k-labels-idx1-ubyte')
        
        train_dataset = MNISTDataset(train_images, train_labels, transform=transform)
        test_dataset = MNISTDataset(test_images, test_labels, transform=transform)
        print("✓ Dataset chargé depuis ./data")
    except FileNotFoundError:
        print("Fichiers non trouvés, téléchargement depuis torchvision...")
        train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
        test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
        print("✓ Dataset téléchargé et chargé")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# ==================== PART 1: CUSTOM CNN ====================
print("\n" + "="*60)
print("PART 1: CNN CLASSIFICATION")
print("="*60)

class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def train_cnn(model, train_loader, test_loader, num_epochs=5):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    start_time = time.time()
    train_losses, test_losses, train_accs, test_accs = [], [], [], []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Testing
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_loss = test_loss / len(test_loader)
        test_acc = correct / total
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%")
    
    training_time = time.time() - start_time
    return model, train_losses, test_losses, train_accs, test_accs, training_time

# Train CNN
cnn_model = CNN().to(device)
cnn_model, cnn_train_loss, cnn_test_loss, cnn_train_acc, cnn_test_acc, cnn_time = train_cnn(cnn_model, train_loader, test_loader, num_epochs=5)

def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, f1

cnn_final_acc, cnn_f1 = evaluate_model(cnn_model, test_loader)
print(f"\nCNN Final Results:")
print(f"Accuracy: {cnn_final_acc:.4f}")
print(f"F1 Score: {cnn_f1:.4f}")
print(f"Training Time: {cnn_time:.2f}s")
print(f"Final Test Loss: {cnn_test_loss[-1]:.4f}")

# ==================== PART 2: FASTER R-CNN ====================
print("\n" + "="*60)
print("PART 2: FASTER R-CNN (Object Detection adapted for Classification)")
print("="*60)

class FasterRCNN_Adapted(nn.Module):
    def __init__(self, num_classes=10):
        super(FasterRCNN_Adapted, self).__init__()
        
        # Backbone: ResNet50 features
        self.backbone = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # RPN-like region classifier
        self.region_classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 1 * 1, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.region_classifier(x)
        return x

def train_fasterrcnn(model, train_loader, test_loader, num_epochs=5):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    start_time = time.time()
    train_losses, test_losses, train_accs, test_accs = [], [], [], []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Testing
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_loss = test_loss / len(test_loader)
        test_acc = correct / total
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%")
    
    training_time = time.time() - start_time
    return model, train_losses, test_losses, train_accs, test_accs, training_time

# Train Faster R-CNN
frcnn_model = FasterRCNN_Adapted().to(device)
frcnn_model, frcnn_train_loss, frcnn_test_loss, frcnn_train_acc, frcnn_test_acc, frcnn_time = train_fasterrcnn(frcnn_model, train_loader, test_loader, num_epochs=5)

frcnn_final_acc, frcnn_f1 = evaluate_model(frcnn_model, test_loader)
print(f"\nFaster R-CNN Final Results:")
print(f"Accuracy: {frcnn_final_acc:.4f}")
print(f"F1 Score: {frcnn_f1:.4f}")
print(f"Training Time: {frcnn_time:.2f}s")
print(f"Final Test Loss: {frcnn_test_loss[-1]:.4f}")

# ==================== PART 3: COMPARISON ====================
print("\n" + "="*60)
print("PART 3: CNN vs FASTER R-CNN COMPARISON")
print("="*60)

comparison_data = {
    'CNN': {
        'Accuracy': cnn_final_acc,
        'F1 Score': cnn_f1,
        'Loss': cnn_test_loss[-1],
        'Time': cnn_time
    },
    'Faster R-CNN': {
        'Accuracy': frcnn_final_acc,
        'F1 Score': frcnn_f1,
        'Loss': frcnn_test_loss[-1],
        'Time': frcnn_time
    }
}

print(f"\n{'Model':<15} {'Accuracy':<15} {'F1 Score':<15} {'Loss':<15} {'Time(s)':<15}")
print("-" * 75)
for model_name, metrics in comparison_data.items():
    print(f"{model_name:<15} {metrics['Accuracy']:.4f}         {metrics['F1 Score']:.4f}         {metrics['Loss']:.4f}         {metrics['Time']:.2f}")

# ==================== PART 4: FINE-TUNING PRE-TRAINED MODELS ====================
print("\n" + "="*60)
print("PART 4: FINE-TUNING VGG16 & ALEXNET")
print("="*60)

# VGG16 Fine-tuning
print("\n--- VGG16 Fine-tuning ---")
vgg16_model = models.vgg16(pretrained=True)

# Freeze early layers
for param in list(vgg16_model.parameters())[:-4]:
    param.requires_grad = False

# Modify classifier for MNIST
vgg16_model.classifier[6] = nn.Linear(4096, 10)
vgg16_model = vgg16_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(filter(lambda p: p.requires_grad, vgg16_model.parameters()), lr=0.0001)

start_time = time.time()
vgg16_train_loss, vgg16_test_loss, vgg16_train_acc, vgg16_test_acc = [], [], [], []

for epoch in range(5):
    vgg16_model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = transforms.Resize((224, 224))(images)  # VGG requires 224x224
        images, labels = images.to(device), labels.to(device)
        
        optimizer_vgg.zero_grad()
        outputs = vgg16_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_vgg.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    vgg16_train_loss.append(train_loss)
    vgg16_train_acc.append(train_acc)
    
    vgg16_model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = transforms.Resize((224, 224))(images)
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16_model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_loss = test_loss / len(test_loader)
    test_acc = correct / total
    vgg16_test_loss.append(test_loss)
    vgg16_test_acc.append(test_acc)
    
    print(f"Epoch [{epoch+1}/5] - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%")

vgg16_time = time.time() - start_time
vgg16_final_acc, vgg16_f1 = evaluate_model(vgg16_model, test_loader)
print(f"\nVGG16 Final Results:")
print(f"Accuracy: {vgg16_final_acc:.4f}")
print(f"F1 Score: {vgg16_f1:.4f}")
print(f"Training Time: {vgg16_time:.2f}s")
print(f"Final Test Loss: {vgg16_test_loss[-1]:.4f}")

# AlexNet Fine-tuning
print("\n--- AlexNet Fine-tuning ---")
alexnet_model = models.alexnet(pretrained=True)

# Freeze early layers
for param in list(alexnet_model.parameters())[:-4]:
    param.requires_grad = False

# Modify classifier
alexnet_model.classifier[6] = nn.Linear(4096, 10)
alexnet_model = alexnet_model.to(device)

optimizer_alex = optim.Adam(filter(lambda p: p.requires_grad, alexnet_model.parameters()), lr=0.0001)

start_time = time.time()
alex_train_loss, alex_test_loss, alex_train_acc, alex_test_acc = [], [], [], []

for epoch in range(5):
    alexnet_model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = transforms.Resize((224, 224))(images)
        images, labels = images.to(device), labels.to(device)
        
        optimizer_alex.zero_grad()
        outputs = alexnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_alex.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    alex_train_loss.append(train_loss)
    alex_train_acc.append(train_acc)
    
    alexnet_model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = transforms.Resize((224, 224))(images)
            images, labels = images.to(device), labels.to(device)
            outputs = alexnet_model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_loss = test_loss / len(test_loader)
    test_acc = correct / total
    alex_test_loss.append(test_loss)
    alex_test_acc.append(test_acc)
    
    print(f"Epoch [{epoch+1}/5] - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%")

alex_time = time.time() - start_time
alex_final_acc, alex_f1 = evaluate_model(alexnet_model, test_loader)
print(f"\nAlexNet Final Results:")
print(f"Accuracy: {alex_final_acc:.4f}")
print(f"F1 Score: {alex_f1:.4f}")
print(f"Training Time: {alex_time:.2f}s")
print(f"Final Test Loss: {alex_test_loss[-1]:.4f}")

# ==================== FINAL COMPARISON ====================
print("\n" + "="*60)
print("FINAL COMPARISON: ALL 4 MODELS")
print("="*60)

final_results = {
    'CNN': {'Accuracy': cnn_final_acc, 'F1': cnn_f1, 'Loss': cnn_test_loss[-1], 'Time': cnn_time},
    'Faster R-CNN': {'Accuracy': frcnn_final_acc, 'F1': frcnn_f1, 'Loss': frcnn_test_loss[-1], 'Time': frcnn_time},
    'VGG16': {'Accuracy': vgg16_final_acc, 'F1': vgg16_f1, 'Loss': vgg16_test_loss[-1], 'Time': vgg16_time},
    'AlexNet': {'Accuracy': alex_final_acc, 'F1': alex_f1, 'Loss': alex_test_loss[-1], 'Time': alex_time}
}

print(f"\n{'Model':<15} {'Accuracy':<15} {'F1 Score':<15} {'Loss':<15} {'Time(s)':<15}")
print("-" * 75)
for model_name, metrics in final_results.items():
    print(f"{model_name:<15} {metrics['Accuracy']:.4f}         {metrics['F1']:.4f}         {metrics['Loss']:.4f}         {metrics['Time']:.2f}")

# ==================== VISUALIZATION ====================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# CNN Loss
axes[0, 0].plot(cnn_train_loss, label='Train', marker='o')
axes[0, 0].plot(cnn_test_loss, label='Test', marker='s')
axes[0, 0].set_title('CNN - Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid()

# Faster R-CNN Loss
axes[0, 1].plot(frcnn_train_loss, label='Train', marker='o')
axes[0, 1].plot(frcnn_test_loss, label='Test', marker='s')
axes[0, 1].set_title('Faster R-CNN - Loss', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid()

# VGG16 Loss
axes[1, 0].plot(vgg16_train_loss, label='Train', marker='o')
axes[1, 0].plot(vgg16_test_loss, label='Test', marker='s')
axes[1, 0].set_title('VGG16 - Loss', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid()

# AlexNet Loss
axes[1, 1].plot(alex_train_loss, label='Train', marker='o')
axes[1, 1].plot(alex_test_loss, label='Test', marker='s')
axes[1, 1].set_title('AlexNet - Loss', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid()

plt.tight_layout()
plt.savefig('all_models_comparison.png', dpi=100)
plt.show()

print("\n✓ Training complete!")